In [17]:
import pandas as pd
from pyproj import Transformer
import matplotlib as plt
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap

In [27]:
mmissed_dedup = pd.read_csv('mmissed_dedup.csv')

In [28]:
mmissed_dedup.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,address_norm,date_only,new_event,event_id,event_date,event_seq,fine_event,first_event
0,79395,2018-06-29,Trash - Curbside/Alley Missed Pickup,the entire street was missed,1 BELLE FORREST AVE C,37206,METRO,9502.0,7.0,1.751942e+06,677895.310490,1 BELLE FORREST AVE C,2018-06-29,True,1,2018-06-29,1,0,True
1,79884,2018-07-02,Trash - Curbside/Alley Missed Pickup,Missed entire street- carts are curbside in fr...,10 Belle Forrest Ave,37206,METRO,9502.0,7.0,1.751718e+06,678077.934103,10 BELLE FORREST AVE,2018-07-02,True,1,2018-07-02,1,0,True
2,257588,2019-10-10,Trash - Curbside/Alley Missed Pickup,miss. driveway is on 14th,"100 Marshall Ct, Nashville, TN 37212, United S...",37212,METRO,9407.0,19.0,1.735543e+06,660440.744955,100 MARSHALL CT,2019-10-10,True,1,2019-10-10,1,0,True
3,266788,2019-11-01,Trash - Curbside/Alley Missed Pickup,missed 102 marshall court as well,"100 Marshall Ct, Nashville, TN 37212, United S...",37212,METRO,9407.0,19.0,1.735543e+06,660440.744955,100 MARSHALL CT,2019-11-01,True,2,2019-11-01,2,200,True
4,253133,2019-09-30,Trash - Curbside/Alley Missed Pickup,MISSED- ALLEY,"1000 Gilmore Ave, Nashville, TN 37204, United ...",37204,METRO,9403.0,17.0,1.736212e+06,652995.000671,1000 GILMORE AVE,2019-09-30,True,1,2019-09-30,1,0,True


In [30]:
mmissed_dedup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3046 entries, 0 to 3045
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    3046 non-null   int64  
 1   Date Opened       3046 non-null   object 
 2   Request           3046 non-null   object 
 3   Description       3042 non-null   object 
 4   Incident Address  3046 non-null   object 
 5   Zip Code          3046 non-null   object 
 6   Trash Hauler      3046 non-null   object 
 7   Trash Route       2996 non-null   float64
 8   Council District  3044 non-null   float64
 9   State Plan X      3045 non-null   float64
 10  State Plan Y      3045 non-null   float64
 11  address_norm      3046 non-null   object 
 12  date_only         3046 non-null   object 
 13  new_event         3046 non-null   bool   
 14  event_id          3046 non-null   int64  
 15  event_date        3046 non-null   object 
 16  event_seq         3046 non-null   int64  


In [31]:
mmissed_dedup['State Plan X'] = pd.to_numeric(mmissed_dedup['State Plan X'], errors = 'coerce')
mmissed_dedup['State Plan Y'] = pd.to_numeric(mmissed_dedup['State Plan Y'], errors = 'coerce')
mmissed_dedup = mmissed_dedup.dropna(subset=['State Plan X', 'State Plan Y'])

In [32]:
transformer = Transformer.from_crs(2274, 4326, always_xy=True)

In [33]:
lons, lats = transformer.transform(mmissed_dedup['State Plan X'].values, mmissed_dedup['State Plan Y'].values)
mmissed_dedup["Longitude"] = lons
mmissed_dedup["Latitude"]  = lats

In [34]:
valid = (mmissed_dedup["Latitude"].between(34, 38)) & (mmissed_dedup["Longitude"].between(-92, -80))
mmissed_dedup = mmissed_dedup[valid].copy()

In [35]:
mmissed_dedup

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,...,address_norm,date_only,new_event,event_id,event_date,event_seq,fine_event,first_event,Longitude,Latitude
0,79395,2018-06-29,Trash - Curbside/Alley Missed Pickup,the entire street was missed,1 BELLE FORREST AVE C,37206,METRO,9502.0,7.0,1.751942e+06,...,1 BELLE FORREST AVE C,2018-06-29,True,1,2018-06-29,1,0,True,-86.733905,36.193409
1,79884,2018-07-02,Trash - Curbside/Alley Missed Pickup,Missed entire street- carts are curbside in fr...,10 Belle Forrest Ave,37206,METRO,9502.0,7.0,1.751718e+06,...,10 BELLE FORREST AVE,2018-07-02,True,1,2018-07-02,1,0,True,-86.734668,36.193906
2,257588,2019-10-10,Trash - Curbside/Alley Missed Pickup,miss. driveway is on 14th,"100 Marshall Ct, Nashville, TN 37212, United S...",37212,METRO,9407.0,19.0,1.735543e+06,...,100 MARSHALL CT,2019-10-10,True,1,2019-10-10,1,0,True,-86.789003,36.145113
3,266788,2019-11-01,Trash - Curbside/Alley Missed Pickup,missed 102 marshall court as well,"100 Marshall Ct, Nashville, TN 37212, United S...",37212,METRO,9407.0,19.0,1.735543e+06,...,100 MARSHALL CT,2019-11-01,True,2,2019-11-01,2,200,True,-86.789003,36.145113
4,253133,2019-09-30,Trash - Curbside/Alley Missed Pickup,MISSED- ALLEY,"1000 Gilmore Ave, Nashville, TN 37204, United ...",37204,METRO,9403.0,17.0,1.736212e+06,...,1000 GILMORE AVE,2019-09-30,True,1,2019-09-30,1,0,True,-86.786533,36.124675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,149959,2019-03-04,Trash - Curbside/Alley Missed Pickup,entire street was missed,"961 16th Ave N, Nashville, TN 37208, United St...",37208,METRO,9208.0,19.0,1.731877e+06,...,961 16TH AVE N,2019-03-04,True,1,2019-03-04,1,0,True,-86.801639,36.166839
3042,224867,2019-08-02,Trash - Curbside/Alley Missed Pickup,ENTIRE STREET AND SURROUNDING STREETS SUCH AS ...,"965 9th Ave S, Nashville, TN 37203, United States",37203,METRO,9407.0,17.0,1.737999e+06,...,965 9TH AVE S,2019-08-02,True,1,2019-08-02,1,0,True,-86.780687,36.145343
3043,226241,2019-08-06,Trash - Curbside/Alley Missed Pickup,I called 311 on 2 AUgust and filed a missed ga...,"965 9th Ave S, Nashville, TN 37203, United States",37203,METRO,9407.0,17.0,1.737999e+06,...,965 9TH AVE S,2019-08-06,False,1,2019-08-02,1,0,False,-86.780687,36.145343
3044,128671,2019-01-07,Trash - Curbside/Alley Missed Pickup,Missed...along with neighbors.,"994 Dozier Pl, Nashville, TN 37216, United States",37216,METRO,9507.0,5.0,1.748996e+06,...,994 DOZIER PL,2019-01-07,True,1,2019-01-07,1,0,True,-86.743980,36.203008


In [36]:
m = folium.Map(location=[36.1627, -86.7816], zoom_start=11, tiles="cartodbpositron")
points = mmissed_dedup[["Latitude", "Longitude"]].values.tolist()
HeatMap(points, radius=12, blur=18, min_opacity=0.25).add_to(m)

m